<h1 style="font-size: 74px">SFLKit: <br>A Workbench for Statistical Fault Localization</h1>

<div style="float: left; font-size:30px"><b>Marius Smytzek</b><br>marius.smytzek@cispa.de</div><div style="float: left;margin-left: 100px; font-size:30px"><b>Andreas Zeller</b><br>zeller@cispa.de</div>

[start](#Statistical-Fault-Localization)

## What is SFLKit

SFLKit is a workbench for statistical fault localization. It comes with the fundamental concepts of statistical debugging and spectrum-based fault localization.

You can use SFLKit out-of-the-box by integrating its command-line interface `sfl.py` or as a library, as we do in this demonstration. We designed SFLKit to be highly configurable and expandable with novel concepts.

To install SFLKit execute

In [1]:
#%pip install .

In [2]:
import enum
import importlib
import inspect
import os
import shutil

from IPython.display import HTML

from sflkit import *
from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

from avicenna import *

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## A faulty Program

First, we need a faulty program. We chose an implementation of the `middle(x, y, z)` function that returns the *middle* number of its three arguments. For example, `middle(1, 3, 2)` should return 2 because `1 < 2` and `2 < 3`. We introduced a fault in this implementation of `middle` that occurs in line 7 `m = y`. 

In [3]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = x 
    else:
        if x > y:
            m = y
        elif x > z:
            m = x # desired line
    return m

Next, we introduce a class to capture test runs' results efficiently. The `TestResult` is an enum with two possible values, `PASS`and `FAIL`. `PASS` donates a passing test case and `FAIL` a failing one.

In [4]:
class TestResult(enum.Enum):
    
    def __repr__(self):
        return self.value
    
    PASS = 'PASS'
    FAIL = 'FAIL'

Now we implement a test function that takes the three arguments of `middle(x, y, z)` and an expected result. This test function compares the return of `middle(x, y, z)` with the desired value and returns `PASS` if they match and `FAIL` otherwise.

In [5]:
def test(function, x, y, z, expected):
    try:
        if function(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL

def test_middle(x, y, z, expected):
    return test(middle, x, y, z, expected)

Let's check the results for some combinations of the numbers 1, 2, and 3. The expected value is in all cases 2.

In [6]:
test_middle(3, 2, 1, expected=2)

PASS

In [7]:
test_middle(3, 1, 2, expected=2)

PASS

In [8]:
test_middle(2, 1, 3, expected=2)

PASS

As you can see, the result of `middle(2, 1, 3)` does not match the expected value 2. Hence, we found a failing test case

## Statistical Fault Localization

Statistical fault localization aims at detecting execution features that correlate with failures, such as whether individual lines are part of the execution.

With the example from above and the three test cases, we can run a statistical fault localization by marking which test case executes which lines:

<table>
<tr>
<td></td>
<td></td>
<td>

```python
middle(3, 2, 1)
```
            
</td>
<td>

```python
middle(3, 1, 2)
```
            
</td>
<td>

```python
middle(2, 1, 3)
```
            
</td>
</tr>
<tr>
<td>

```
1
2
3
4
5
6
7
8
9
10
11
12
13
```
            
</td>
<td>

```python
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m
```
            
</td>
<td>

```

       X
       X





       X
       X


       X
```
            
</td>
<td>

```

       X
       X
       X
       
       X






       X
```
         
</td>
<td>

```

       X
       X
       X
       
       X
       X





       X
```
            
</td>
</tr>
  
</table>

We can see here that line 7 gets only executed by the failing test and not by the passing tests, making it the most likely line to contain the fault, which we already know is true.

[next](#Configuring-SFLKit)

## Instrument the Program

Subsequently, we want to leverage SFLKit to find the location in the code that is most likely to include the fault.

Let us first get the source of our function and write it to a file so we have something to perform our instrumentation and analysis.

We leverage Python's `inspect` to get the source code.

In [9]:
source = inspect.getsource(middle)
print(source)

def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = x 
    else:
        if x > y:
            m = y
        elif x > z:
            m = x # desired line
    return m



We also define the file we write the source to and the python file we will work on, namely `middle.py` and `tmp.py`, respectively.

In [10]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'

We write the source code to `middle.py`.

In [11]:
with open(middle_py, 'w') as fp:
    fp.write(source)

Let's update our test function to import the correct module and run the `middle(x, y, z)` from this module.

In [12]:
def test_middle_import(x, y, z, expected):
    from middle import middle
    return test(middle, x, y, z, expected)

We repeat the tests to check that our setup works with the import.

In [13]:
test_middle_import(3, 2, 1, expected=2), test_middle_import(3, 1, 2, expected=2), test_middle_import(2, 1, 3, expected=2)

(PASS, PASS, PASS)

We produced the same results for the test cases, so it seems to work.

### Configuring SFLKit

The `Config` class provides comfortable access to `SFLKit` by defining the fundamental concepts we want to investigate.

We give some information for the config that we need to define. First, we need the path to the source we want to investigate, which we already have in `middle_py`. Next, we need an out, `tmp_py`. We also need:

The language of our subject is `'python'`.
Let's start with `'line'` as the predicates we want to investigate.
We define `'tarantula'` as our evaluation metric for the predicates, i.e., the similarity coefficient.
We also need a list of passing and failing tests used during the analysis.

In [14]:
language='python'
predicates='line'
metrics='Tarantula'
passing='event-files/0,event-files/1'
failing='event-files/2'

We define a function that gives as a `Config` object, so we do not need to create it manually every time we change something.

In [15]:
def get_config():
    return Config.create(path=middle_py, working=tmp_py, language=language, predicates=predicates, metrics=metrics, passing=passing, failing=failing)

Now we can define a function that instruments our subject. We leverage `SFLKit`'s `instrument_config()`, which takes a config we create with our defined `get_config()` and instruments the subject. We can also show the content of the instrumented python file with this function.

In [16]:
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

Now we instrument our `middle.py` subject and check the results.

In [17]:
instrument()

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 0 events in middle.py.


import sflkitlib.lib


def middle(x, y, z):
    sflkitlib.lib.add_line_event('middle.py', 2, 0)
    m = z
    sflkitlib.lib.add_line_event('middle.py', 3, 1)
    if y < z:
        sflkitlib.lib.add_line_event('middle.py', 4, 2)
        if x < y:
            sflkitlib.lib.add_line_event('middle.py', 5, 3)
            m = y
        else:
            sflkitlib.lib.add_line_event('middle.py', 6, 4)
            if x < z:
                sflkitlib.lib.add_line_event('middle.py', 7, 5)
                m = x
    else:
        sflkitlib.lib.add_line_event('middle.py', 9, 6)
        if x > y:
            sflkitlib.lib.add_line_event('middle.py', 10, 7)
            m = y
        else:
            sflkitlib.lib.add_line_event('middle.py', 11, 8)
            if x > z:
                sflkitlib.lib.add_line_event('middle.py', 12, 9)
                m = x
    sflkitlib.lib.add_line_event('middle.py', 13, 10)
    return m



As you can see, the instrumentation added an import at the beginning to a lib that comes with `SFLKit`, cluing the execution of files together. Moreover, the instrumentation added a function call function of the lib in front of each executable line that tracks the executed lines.

## Get and Analyze Events

Now, we want to extract the events from the execution of tests. Therefore, we need to adjust our test execution function again because the shared library for tracking the events does not know when to start and stop. We need to reset this library before entering our `middle.py` and tell the library to dump the events after the function finishes.

In [18]:
def test_tmp(x, y, z, expected): 
    import tmp
    importlib.reload(tmp)
    tmp.sflkitlib.lib.reset()
    try:
        return test(tmp.middle, x, y, z, expected)
    finally:
        tmp.sflkitlib.lib.dump_events()
        del tmp

In [19]:
def test_pls_work(x, y, z,expected): 
    import tmp
    importlib.reload(tmp)
    tmp.sflkitlib.lib.reset()
    test(tmp.middle, x, y, z, expected)
    tmp.sflkitlib.lib.dump_events()
    del tmp

We define a path to write the generated event logs.

In [20]:
event_files = 'event-files'

Then, we need a function to generate the event log from the previous test cases. We change the environment variable `EVENTS_PATH` to the output path of the event log file before running each test.

In [21]:
# def run_tests():
#     if os.path.exists(event_files):
#         shutil.rmtree(event_files)
#     os.mkdir(event_files)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '0')
#     test_tmp(3, 2, 1, expected=2)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '1')
#     test_tmp(3, 1, 2, expected=2)
#     os.environ['EVENTS_PATH'] = os.path.join(event_files, '2')
#     test_tmp(2, 1, 3, expected=2)

With this, we can execute the tests and analyze the result with the help of `analyze_config()` from SFLKit.

In [22]:
# def analyze():
#     run_tests()
#     return analyze_config(get_config())

Let's execute the tests and analyze the event logs for lines and the Tarantula metric.

In [23]:
#results = analyze()

The results look something like this:

In [24]:
#results

This structure maps analysis objects and metrics to a list of sorted suggestions where the fault occurs.

Now, we can put all this together and produce a pretty output that shows us where the fault originates by leveraging `SFLKit`'s `ColorCode` object.

As you can see, the analysis indeed suggests the buggy line 7 as the most suspicious.

But what if lines are not enough to show the fault?

What if the metric we have chosen for evaluation is insufficient?

### Extension using Avicenna
We seek to utilize SFLKit's instrmentation of program code line events to determine constraints describing a specific line's reachability with Avicenna. 

1. We must instrument our test program, middle, and use that output.
2. We access the event files and check what line was called, oracle determines YES or NO.
3. If yes, learn from that event file, make constraint loop with Avicenna
4. If no, call other event files to check for line event YES.


Instrumentation for line events is coded as follows: (event_type, file_name, line_num, event_id)


First we need to instrument our event files for a given program or function.

In [25]:
def run_test_and_instrument(middle_input, event_files_path, iterator):
    
    os.environ['EVENTS_PATH'] = os.path.join(event_files_path, str(iterator))
    
    expected_return = sorted(middle_input)[1]
    test_tmp(middle_input[0],
             middle_input[1],
             middle_input[2],
             expected=expected_return) # instrumenting middle

The oracle has to be able to determine if a given line is called when a test case is run. This means we need to call sflkit's instrumentation for each test case when determining whether it is valuable and can be used for finding constraints that lead to the desired line.

In [26]:
import string
import csv
import logging

from pathlib import Path
from avicenna import Avicenna
from avicenna.oracle import OracleResult
from isla.solver import ISLaSolver



In [27]:
# This oracle has to run sflkit and create event_files, as well as then read through them and ideally delete as well

# for now just use bug for line event called, add TRUE/FALSE later
def prototype_oracle_line_event(event_file_path):
    
    #is file path valid?
    if event_file_path.exists() == False or event_file_path.is_dir() == True:
        # add error handling 
        return OracleResult.UNDEF
    
    # check if desired_line was passed as int and not as str
        # QUESTION should we only accept str to begin with? check for numbers only?
    #if desired_line.type() == int:
    #   desired_line = str(desired_line)
    
    event_file = event_file_path.open() # replace this with the event file instrumentation
    
    with open(event_file_path, newline='') as event_file:
        
        # split each line at , to get columns separated
        for line in event_file.readlines():
            reached_line = line.split(',')
            #lines_reached.add(peep[-2])
            if reached_line[-2] == '12':
                return OracleResult.BUG
        
    return OracleResult.NO_BUG

We can now go through our event-files folder and iterate through each file, representing a test case. The relevant test cases will be denoted by an OracleResult.BUG for now. We can use this behavior-triggering input as our input for a run of Avicenna to find the constraints leading to this behavior.

In [28]:
# problems arose with mismatched input types. middle takes int inputs, but the input through avicenna works with 
# strings. therefore we need to convert these types for each function when required.
def oracle_run_test_and_instrument(middle_input, event_files_path, event_file_num):
    
    #if os.path.exists(event_files_path):
    #    shutil.rmtree(event_files_path) # ** can be removed, will collect trash at the end of run ** 
    #os.mkdir(event_files_path) # just needs to happen once before the run now 
    #os.environ['EVENTS_PATH'] = os.path.join(event_files_path, event_file_num)
    
    
    # convert inputs
    middle_input = middle_input.__str__()
    middle_input = middle_input.split(',')
    
    sorted_inputs = [int(middle_input[0]), int(middle_input[1]), int(middle_input[2])]
    sorted_inputs.sort()
    expected = sorted_inputs[1]
    
    test_tmp(int(middle_input[0]), int(middle_input[1]), int(middle_input[2]), expected=expected) # instrumenting middle

In [29]:
# add desired line too 
def oracle_middle_sfl(test_case):
        
    # #is file path valid?
    # if event_file_path.exists() == False or event_file_path.is_dir() == True:
    #     # add error handling 
    #     return OracleResult.UNDEF
    
    # check if desired_line was passed as int and not as str
        # QUESTION should we only accept str to begin with? check for numbers only?
    #if desired_line.type() == int:
    #   desired_line = str(desired_line)
    path = Path('./event-files/')
    oracle_run_test_and_instrument(test_case, path, '0') # replace this with the event file instrumentation
    
    with open('./event-files/0', newline='') as event_file:
        
        # split each line at , to get columns separated
        for line in event_file.readlines():
            reached_line = line.split(',')
            #print(reached_line)
            if reached_line[-2] == '12':   
                return OracleResult.BUG
            
        return OracleResult.NO_BUG 

In [30]:
#print(oracle_middle_sfl('2,3,1').value)

In [31]:
#print(oracle_middle_sfl('8,24,4'))

In [32]:
#print(oracle_middle_sfl('121,2,23'))

In [33]:
#print(oracle_middle_sfl('2,1,3'))

The oracle used in this run of Avicenna will be the oracle checking against line-event calls. This requires us to run sflkit using Avicenna's new hypotheses after every run. 

- TODO : update Avicenna and fix up prototype for it 
- TODO : implement prototype functionality using SFLKit and Avicenna 0.2 

#### Testing inputs for regular middle bug. 

In [34]:
middle_grammar = {
    "<start>": ["<stmt>"],
    "<stmt>": ["<x>,<y>,<z>"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

In [35]:
def oracle_middle(inp):
    inp = str(inp).split(",")
    x = int(inp[0])
    y = int(inp[1])
    z = int(inp[2])
    expected = sorted([x,y,z])[1]
    try:
        if middle(x, y, z) == expected:
            return OracleResult.NO_BUG
        else:
            return OracleResult.BUG
    except BaseException:
        return OracleResult.UNDEF

In [36]:
# 321, 312, 213
middle_inputs = ['2,3,1', '3,1,2', '2,1,3']

In [43]:
path = Path('./event-files/')
if os.path.exists(path):
    shutil.rmtree(path) 
os.mkdir(path) 
os.environ['EVENTS_PATH'] = os.path.join(path, '0')

In [44]:
avicenna = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=oracle_middle_sfl,
    max_iterations=10,
    max_excluded_features=4, 
)

In [47]:
import warnings
# Suppress the specific SHAP warning
warnings.filterwarnings(
    "ignore",
    message="LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray",
)
warnings.filterwarnings(
    "ignore", 
    message="No further splits with positive gain, best gain: -inf"
)

In [48]:


logging.basicConfig(filename='avicenna.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
# only 2 constraints used in the end why
best_invariant = avicenna.explain() # unparse with islaunparse for further use



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 23, number of negative: 120
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 213
[LightGBM] [Info] Number of data points in the train set: 143, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.160839 -> initscore=-1.651998
[LightGBM] [Info] Start training from score -1.651998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

In [39]:
shutil.rmtree(Path('./event-files/'))

In [40]:
solver = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0]
)
print(best_invariant[0])

(∀ elem_1 ∈ start: (∃ elem_2 ∈ start: (StrToInt(elem_1) > StrToInt(elem_2))) ∧ ∀ elem_0 ∈ start: (∃ elem_3 ∈ start: (StrToInt(elem_0) > StrToInt(elem_3))))


In [41]:
# can we use this to fuzz inputs with the constraints? why doesnt it work 
a = 2 

https://github.com/uds-se/sflkit

<img src="qrcode.png" style="width:500px">